# Optimizing your code

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
%conda install numba numexpr cython

When your math works, the next step is often to try and make the code faster.
Optimizing is very delicate work as oftentimes we immediately go for more aggressive options. 
As a rule of thumb takes these steps towards optimisation:
   
   1. See if its been done already
   2. Code the math
   3. Simplify
   4. Algorithmic improvement
   5. Use libraries like `numexpr` and `numba`
   6. Parallelize
   7. Rewrite in C++/FORTRAN

## Quick optimization (Caching)

Caching function results is often a quick and easy way to get some performance out of your code. This is best used on functions with a limited range of arguments. Python provides the `lru_cache` decorator from the `functools` package

In [1]:
def compute_a_slow_result(x):
    import time
    
    time.sleep(0.1)
    return x*2



In [2]:
%timeit compute_a_slow_result(10)

103 ms ± 562 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Lets now decorate our function with `lru_cache`

In [ ]:
from functools import lru_cache

@lru_cache(maxsize=4)
def compute_a_slow_result_cache(x):
    import time
    
    time.sleep(0.1)
    return x*2



In [ ]:
compute_a_slow_result_cache(10)

In [ ]:
%timeit compute_a_slow_result_cache(10)

In the background `lru_cache` is storing a list of function arguments and return values, when a function with an argument is run for the first time it is run normally and then its return stored. The next time the same arguments are used this will instead simply return the store value! the `maxsize=4` tells `lru_cache` to cache a maximum of 4 function calls.

Our function can be broken if we use a random value each time:

In [ ]:
import random
%timeit compute_a_slow_result_cache(random.randint(1,100))

# Test Problem

Lets write our own linterpolation algorithm. Whilst numpy and scipy have their own interpolation. Ours will be slightly different.

Mathematically the formula looks like this:

$$
y = \frac{e^{y_0^{2}}(x_1 - x)+y_1(x - x_0)}{x_1 - x_0}
$$

this has no physics behind it other than taking the linear interpolation algorithm and adding a square exp function.

Lets implement and test this function

In [1]:
def linear_interpolate(y0, y1, x0, x1, x):
    
    return (np.exp(y0**2) * (x1 - x) + y1 * (x - x0))  / (x1 - x0)

In [ ]:
n = 50_000_000
y0 = np.random.rand(n)
y1 = np.random.rand(n)
%timeit linear_interpolate(y0, y1, 2, 3, 2.5)

## Numexpr

With numpy we get pretty decent performance. However we can push this a bit further with minimal effort by using the `numexpr` package.

Numexpr can take advantage of threading and inbuilt libraries to get faster performance. It works by compiling a computational string that represent the computation and figuring out how best to split the computation up.

In [ ]:
def linear_interpolate_numexpr(y0, y1, x0, x1, x):
    import numexpr as ne
    
    return ne.evaluate('(exp(y0**2) * (x1 - x) + y1 * (x - x0))  / (x1 - x0)')

In [ ]:
%timeit linear_interpolate_numexpr(y0, y1, 2, 3, 2.5)

## FORTRAN

The classic programming language. `numpy` actually has the `f2py` module specifically to allow FORTRAN code to be included in Python! We can run it from the command line like this:
    
    python -m numpy.f2py --quiet -c src/interp.f90 -m vect

Where it will create a new python module stated by the last `-m` argument. Lets see it in action!

In [ ]:
import sys
!{sys.executable} -m numpy.f2py --quiet -c src/interp.f90 -m vect  --fcompiler=gnu95 --f90flags=-O3

In [ ]:
import vect
vect.fort_linear?

In [ ]:
%timeit vect.fort_linear(y0, y1, 2, 3, 2.5)

## Cython

In [ ]:
%load_ext cython

In [ ]:
%%cython -a
import numpy as np
cimport numpy as np
ctypedef np.float64_t dtype_t
cimport cython
from libc.math cimport exp


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
@cython.cdivision(True)
def linear_cython(double[::1] y0,
                  double[::1] y1, 
                  double x0, double x1, double x):
    cdef Py_ssize_t i
    
    cdef Py_ssize_t y_max = y0.shape[0]
    
    result = np.zeros(y0.shape[0], dtype=np.float64)
    
    cdef double[::1] result_view = result
    for i in range(y_max):
        result_view[i] = (exp(y0[i] * y0[i]) * (x1 - x) + y1[i] * (x - x0))  / (x1 - x0)
    return result
        
        
        

In [ ]:
%timeit linear_cython(y0,y1,2,3,2.5)

## Numba

In [ ]:
def linear_interpolate_bad(y0, y1, x0, x1, x):
    out = np.zeros(y0.shape)
    for n in numba.prange(y0.shape[0]):
        out[n] = (np.exp(y0[n]**2) * (x1 - x) + y1[n] * (x - x0))  / (x1 - x0)

    return out

In [ ]:
%time linear_interpolate_bad(y0, y1, 2, 3, 2.5)

In [ ]:
import numba

@numba.njit
def linear_interpolate_good(y0, y1, x0, x1, x):
    out = np.zeros(y0.shape)
    for n in numba.prange(y0.shape[0]):
        out[n] = (np.exp(y0[n]**2) * (x1 - x) + y1[n] * (x - x0))  / (x1 - x0)

    return out

In [ ]:
%timeit linear_interpolate_good(y0, y1, 2, 3, 2.5)

In [ ]:
import numba

@numba.njit(parallel=True, fastmath=True)
def linear_interpolate_best(y0, y1, x0, x1, x):
    out = np.zeros(y0.shape)
    for n in numba.prange(y0.shape[0]):
        out[n] = (np.exp(y0[n]**2) * (x1 - x) + y1[n] * (x - x0))  / (x1 - x0)

    return out

In [ ]:
%timeit linear_interpolate_best(y0, y1, 2, 3, 2.5)

Go away FORTRAN and C++ shooo shooooo

In [ ]:
def mandelbrot_python(size, iterations):
    m = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            c = (-2 + 3. / size * j +
                 1j * (1.5 - 3. / size * i))
            z = 0
            for n in range(iterations):
                if np.abs(z) <= 10:
                    z = z * z + c
                    m[i, j] = n
                else:
                    break
    return m

In [ ]:
size = 400
iterations = 100

In [ ]:
%timeit mandelbrot_python(size, iterations)

In [ ]:
@numba.njit
def mandelbrot_numba(size, iterations):
    m = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            c = (-2 + 3. / size * j +
                 1j * (1.5 - 3. / size * i))
            z = 0
            for n in range(iterations):
                if np.abs(z) <= 10:
                    z = z * z + c
                    m[i, j] = n
                else:
                    break
    return m

In [ ]:
%timeit mandelbrot_numba(size, iterations)

In [ ]:
m = mandelbrot_numba(size, iterations)

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(np.log(m), cmap=plt.cm.hot)
ax.set_axis_off()